In [1]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw
import os
import pandas as pd
from datetime import datetime, timezone
import time
import traceback

In [3]:
# Reddit API credentials
reddit = praw.Reddit(
    client_id="a90Z5hn7arSTIjh5cDFZUg",
    client_secret="wV9_DZyb0bcT7hdLljNKzxaSm13Kaw",
    user_agent="Reddit-Comment-Scraper/1.0"
)

In [12]:
# List to store extracted data
"""
"gaming","Games","anime","movies","manga","Music","books","television",
"hiphopheads","Fantasy","comics","TrueOffMyChest","tifu","Marvel","DCcomics",
"netflix","Isekai","OtomeIsekai","nosleep","TrueFilm","MovieDetails","webtoons",
"manhwa","LetsTalkMusic","FanTheories","Filmmakers","Screenwriting","WritingPrompts"
"""
data = []
reddit_title = "gaming"
reddit_titles = ["gaming","Games","anime","movies","manga","Music","books","television"]
fileName = "reddit_comments.csv"

In [5]:
# Function to convert UTC timestamp to human-readable format
def format_timestamp(utc_timestamp):
    return datetime.fromtimestamp(utc_timestamp, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

In [6]:
# Recursive function to fetch subcomments
def fetch_subcomments(subreddit_name,comment, post_title, post_url, parent_id=None):
    for reply in comment.replies:
        data.append({
            "Post Title": post_title,
            "Post URL": post_url,
            "Comment ID": reply.id,
            "Parent ID": parent_id,
            "Author": reply.author,
            "Timestamp": format_timestamp(reply.created_utc),
            "Comment": reply.body,
            "Score": reply.score,
            "Reddit Name": subreddit_name
        })
        fetch_subcomments(subreddit_name,reply, post_title, post_url, reply.id)  # Recursively fetch deeper replies

# Function to fetch and save all comments
def fetch_comments(subreddit_name, post_limit=10):
    print(f"\nFetching comments from r/{subreddit_name}...\n")
    
    subreddit = reddit.subreddit(subreddit_name)
    count = 1
    for submission in subreddit.hot(limit=post_limit):  # Fetch 'hot' posts
        print(f"\n{count} Processing Post: {submission.title}...")
        submission.comments.replace_more(limit=None)  # Load all comments

        for comment in submission.comments:
            data.append({
                "Post Title": submission.title,
                "Post URL": submission.url,
                "Comment ID": comment.id,
                "Parent ID": None,
                "Author": comment.author,
                "Timestamp": format_timestamp(comment.created_utc),
                "Comment": comment.body,
                "Score": comment.score,
                "Reddit Name": subreddit_name
            })
            fetch_subcomments(subreddit_name, comment, submission.title, submission.url, comment.id)  # Fetch subcomments
        count+=1
        time.sleep(30)

In [13]:
# Convert data to a DataFrame
df_new = pd.DataFrame(data)

# Fetch comments from a subreddit
# Change subreddit & limit as needed
try:
    #fetch_comments(reddit_title, post_limit=25)
    for title in reddit_titles:
        try:
            fetch_comments(title, post_limit=50)
            time.sleep(5)
        except Exception as e:
            print(f"⚠️ Skipping post due to error: {e}")
            traceback.print_exc()  # This will show the stack trace
except Exception as e:
    print(f"⚠️ Skipping story due to error: {e}")
    
# Save to CSV
print(f"\n✅ Data saved to {fileName}")


Fetching comments from r/gaming...


1 Processing Post: Making Friends Monday! Share your game tags here!...

2 Processing Post: Former Concord dev says 'it was gut wrenching to see our project fail' but urges players not to judge Marathon too soon: 'To call this game a failure before it’s even out is wild to me'...

3 Processing Post: What's the Most Ridiculous Name You've Seen for Premium Currency in a Game?...

4 Processing Post: Game console button layout...

5 Processing Post: Dragon Age writer David Gaider claims Mass Effect got preferential treatment over Dragon Age....

6 Processing Post: What game have you played that had such a good story that it made you have genuine post gameplay depression?...

7 Processing Post: Almost 10 years later, and i still feel like a monster....

8 Processing Post: Metro 2033 Redux is free to keep on Steam if you claim it in time...

9 Processing Post: Are there any games with realistic damage? I.E. If you headshot a boss you kill it in one hit?.

In [16]:
csv_data = pd.read_csv(fileName)
csv_data_1 = pd.read_csv("reddit_comments - Copy.csv")
df_new = pd.DataFrame(data)
print(len(data),len(csv_data), len(df_new), len(csv_data_1))

71730 106833 71730 14173


In [15]:
combined_df = pd.concat([df_new, csv_data], ignore_index=True)
combined_df.to_csv(fileName, index=False, encoding="utf-8")
len(combined_df)

106833